# Exercise-1: Training Deep Neural Network on MNIST

Train a controlled deep neural network on the MNIST dataset. Set random seeds to 42.
Load and preprocess MNIST. Build the network using the following configuration:
* Flatten input images to 28 × 28 = 784 features
* 3 hidden layers, 64 neurons each
* ELU activation function
* He normal initialization
* Output layer: 10 neurons with softmax
* Optimizer: Nadam
* learning rate = 0.001, loss=sparse categorical crossentropy
* EarlyStopping callback: monitor validation loss, patience = 5, restore best weights
* epochs = 50, batch size = 32
* Use only the first 1000 training samples and first 200 test samples

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import random

In [2]:
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset

In [3]:
# 1. Set seeds

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# 2. Load MNIST (only part)

transform = transforms.ToTensor()

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_subset = Subset(train_dataset, range(1000))   # first 1000 samples
test_subset  = Subset(test_dataset, range(200))     # first 200 samples

batch_size = 32
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

# 3. Build model

class MNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 64),
            nn.ELU(),
            nn.Linear(64, 64),
            nn.ELU(),
            nn.Linear(64, 64),
            nn.ELU(),
            nn.Linear(64, 10)  # logits -> CrossEntropyLoss will softmax
        )
        self.init_weights()

    def init_weights(self):
        for layer in self.net:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_normal_(layer.weight)  # He normal init
                nn.init.zeros_(layer.bias)

    def forward(self, x):
        return self.net(x)

model = MNISTModel()

# 4. Optimizer & loss

optimizer = optim.NAdam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 5. Training with Early Stopping

epochs = 50
patience = 5
best_loss = np.inf
patience_counter = 0
best_state = None

for epoch in range(epochs):
    model.train()
    train_loss = 0

    for x, y in train_loader:
        optimizer.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # valdiation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x, y in val_loader:
            preds = model(x)
            loss = criterion(preds, y)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    train_loss /= len(train_loader)

    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    # Early stopping
    if val_loss < best_loss:
        best_loss = val_loss
        best_state = model.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

# Restore best weights
model.load_state_dict(best_state)
print("Restored best model weights.")

100%|██████████| 9.91M/9.91M [00:00<00:00, 18.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 537kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.56MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.70MB/s]


Epoch 01 | Train Loss: 1.4510 | Val Loss: 0.8960
Epoch 02 | Train Loss: 0.5706 | Val Loss: 0.5384
Epoch 03 | Train Loss: 0.3566 | Val Loss: 0.4145
Epoch 04 | Train Loss: 0.2649 | Val Loss: 0.3699
Epoch 05 | Train Loss: 0.1881 | Val Loss: 0.3231
Epoch 06 | Train Loss: 0.1341 | Val Loss: 0.3018
Epoch 07 | Train Loss: 0.1017 | Val Loss: 0.3012
Epoch 08 | Train Loss: 0.0732 | Val Loss: 0.2884
Epoch 09 | Train Loss: 0.0528 | Val Loss: 0.2864
Epoch 10 | Train Loss: 0.0383 | Val Loss: 0.2985
Epoch 11 | Train Loss: 0.0275 | Val Loss: 0.3177
Epoch 12 | Train Loss: 0.0225 | Val Loss: 0.3262
Epoch 13 | Train Loss: 0.0171 | Val Loss: 0.3215
Epoch 14 | Train Loss: 0.0143 | Val Loss: 0.3109
Early stopping triggered.
Restored best model weights.


### Q1.1 Report the obtained test accuracy

In [5]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for x, y in val_loader:
        preds = model(x).argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

print(f"Accuracy: {correct/total:.2%}")


Accuracy: 92.00%


# Exercise-2: Training Deep Neural Network on CIFAR-10
Train a controlled deep neural network on the CIFAR-10 dataset. Set random seeds to
42. Load and preprocess CIFAR-10. Build the network using the following configuration:
* Flatten input images to 32 × 32 × 3 = 3072 features
* 4 hidden layers, 256 neurons each
* ELU activation function
* He normal initialization
* Output layer: 10 neurons with softmax
* Optimizer: Nadam
* learning rate = 0.001, loss =′ sparse categorical crossentropy′
* EarlyStopping callback: monitor validation loss, patience = 5, restore best weights
* epochs = 50, batch size = 128
* Use only the first 5000 training samples and first 1000 test samples

In [6]:
# 1. Set random (already done above)
# 2. Load CIFAR-10 (only part)
transform = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset  = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_subset = Subset(train_dataset, range(5000))   # first 5000 samples
test_subset  = Subset(test_dataset, range(1000))    # first 1000 samples

batch_size = 128
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

# 3. Build Model

class CIFAR10MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3072, 256),
            nn.ELU(),
            nn.Linear(256, 256),
            nn.ELU(),
            nn.Linear(256, 256),
            nn.ELU(),
            nn.Linear(256, 256),
            nn.ELU(),
            nn.Linear(256, 10)  # logits (softmax done in CrossEntropy)
        )
        self.init_weights()

    def init_weights(self):
        for layer in self.model:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_normal_(layer.weight)  # He init
                nn.init.zeros_(layer.bias)

    def forward(self, x):
        return self.model(x)

model = CIFAR10MLP()

# 4. Optimizer & loss

optimizer = optim.NAdam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 5. Train w/ Early Stopping

epochs = 50
patience = 5
best_loss = np.inf
patience_counter = 0
best_state = None

for epoch in range(epochs):
    model.train()
    train_loss = 0

    for x, y in train_loader:
        optimizer.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x, y in val_loader:
            preds = model(x)
            loss = criterion(preds, y)
            val_loss += loss.item()

    train_loss /= len(train_loader)
    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    # Early stopping logic
    if val_loss < best_loss:
        best_loss = val_loss
        best_state = model.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

# Restore best weights
model.load_state_dict(best_state)
print("Restored best model weights.")

100%|██████████| 170M/170M [00:05<00:00, 33.9MB/s]


Epoch 01 | Train Loss: 3.2324 | Val Loss: 2.7083
Epoch 02 | Train Loss: 2.1901 | Val Loss: 2.4268
Epoch 03 | Train Loss: 2.0484 | Val Loss: 2.7258
Epoch 04 | Train Loss: 2.0169 | Val Loss: 3.0117
Epoch 05 | Train Loss: 1.9854 | Val Loss: 2.8556
Epoch 06 | Train Loss: 1.9054 | Val Loss: 2.3269
Epoch 07 | Train Loss: 1.8354 | Val Loss: 2.6527
Epoch 08 | Train Loss: 1.8477 | Val Loss: 2.8217
Epoch 09 | Train Loss: 1.9537 | Val Loss: 2.2374
Epoch 10 | Train Loss: 1.7636 | Val Loss: 2.6508
Epoch 11 | Train Loss: 1.7576 | Val Loss: 2.9835
Epoch 12 | Train Loss: 1.7567 | Val Loss: 3.0352
Epoch 13 | Train Loss: 1.7366 | Val Loss: 2.0966
Epoch 14 | Train Loss: 1.6261 | Val Loss: 2.3295
Epoch 15 | Train Loss: 1.6397 | Val Loss: 2.3083
Epoch 16 | Train Loss: 1.5778 | Val Loss: 3.0284
Epoch 17 | Train Loss: 1.6641 | Val Loss: 2.2313
Epoch 18 | Train Loss: 1.5161 | Val Loss: 4.8174
Early stopping triggered.
Restored best model weights.


### Q2.1 Report the obtained the test accuracy.

In [7]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for x, y in val_loader:
        preds = model(x).argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

print(f"Final Test Accuracy: {correct/total:.2%}")


Final Test Accuracy: 12.90%


# Exercise-3: Regularization with Alpha Dropout and MC Dropout
Using the MNIST dataset, extend the previously trained deep neural network by applying
Alpha Dropout. Then, without retraining, use Monte Carlo (MC) Dropout at inference
to estimate if you can achieve better accuracy. Set random seeds to 42. Use the following
configuration:
* Flatten input images to 28 × 28 = 784 features
* 3 hidden layers, 64 neurons each
* SELU activation function (required for Alpha Dropout)
* LeCun normal initialization
* Alpha Dropout rate: 0.1 in all hidden layers
* Output layer: 10 neurons with softmax
* Optimizer: Nadam
* learning rate = 0.001, loss=sparse categorical crossentropy
* epochs = 50, batch size = 32
* Use only the first 1000 training samples and first 200 test samples
* For MC Dropout, enable dropout during inference and average predictions over 20 stochastic forward passes

In [8]:
# 2. Load MNIST
# -------------------------
transform = transforms.ToTensor()
train_ds = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_ds  = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False)

# -------------------------
# 3. Define Model (SELU + AlphaDropout)
# -------------------------
class MNISTAlphaDropoutNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 64),
            nn.SELU(),
            nn.AlphaDropout(p=0.1),
            nn.Linear(64, 64),
            nn.SELU(),
            nn.AlphaDropout(p=0.1),
            nn.Linear(64, 64),
            nn.SELU(),
            nn.AlphaDropout(p=0.1),
            nn.Linear(64, 10)  # logits
        )
        self.init_lecun_normal()

    def init_lecun_normal(self):
        for layer in self.model:
            if isinstance(layer, nn.Linear):
                fan_in = layer.weight.shape[1]
                nn.init.normal_(layer.weight, mean=0.0, std=np.sqrt(1.0 / fan_in))
                nn.init.zeros_(layer.bias)

    def forward(self, x):
        return self.model(x)

model = MNISTAlphaDropoutNet()

# -------------------------
# 4. Optimizer & loss
# -------------------------
optimizer = optim.NAdam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# -------------------------
# 5. Train normally
# -------------------------
epochs = 50
for epoch in range(epochs):
    model.train()
    train_loss = 0

    for x, y in train_loader:
        optimizer.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # valdiation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x, y in val_loader:
            preds = model(x)
            loss = criterion(preds, y)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    train_loss /= len(train_loader)

    print(f"Epoch {epoch+1:02d} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    # Early stopping
    if val_loss < best_loss:
        best_loss = val_loss
        best_state = model.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break

print("Training complete.")

# -------------------------
# 6. Standard accuracy (dropout OFF)
# -------------------------
def evaluate(model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in test_loader:
            preds = model(x).argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)

    return correct / total


Epoch 01 | Loss: 0.4412
Epoch 02 | Loss: 0.2275
Epoch 03 | Loss: 0.1765
Epoch 04 | Loss: 0.1456
Epoch 05 | Loss: 0.1287
Epoch 06 | Loss: 0.1149
Epoch 07 | Loss: 0.1068
Epoch 08 | Loss: 0.0976
Epoch 09 | Loss: 0.0902
Epoch 10 | Loss: 0.0858
Epoch 11 | Loss: 0.0815
Epoch 12 | Loss: 0.0781
Epoch 13 | Loss: 0.0736
Epoch 14 | Loss: 0.0712
Epoch 15 | Loss: 0.0688
Epoch 16 | Loss: 0.0673
Epoch 17 | Loss: 0.0634
Epoch 18 | Loss: 0.0620
Epoch 19 | Loss: 0.0608
Epoch 20 | Loss: 0.0585
Epoch 21 | Loss: 0.0569
Epoch 22 | Loss: 0.0539
Epoch 23 | Loss: 0.0557
Epoch 24 | Loss: 0.0513
Epoch 25 | Loss: 0.0526
Epoch 26 | Loss: 0.0512
Epoch 27 | Loss: 0.0508
Epoch 28 | Loss: 0.0475
Epoch 29 | Loss: 0.0471
Epoch 30 | Loss: 0.0487
Epoch 31 | Loss: 0.0455
Epoch 32 | Loss: 0.0463
Epoch 33 | Loss: 0.0448
Epoch 34 | Loss: 0.0435
Epoch 35 | Loss: 0.0443
Epoch 36 | Loss: 0.0439
Epoch 37 | Loss: 0.0433
Epoch 38 | Loss: 0.0410
Epoch 39 | Loss: 0.0404
Epoch 40 | Loss: 0.0419
Epoch 41 | Loss: 0.0411
Epoch 42 | Loss:

## Q3.1 Report the test accuracy of the network with Alpha Dropout applied during training

In [9]:
base_acc = evaluate(model)
print(f"Standard Test Accuracy (Dropout OFF): {base_acc:.4%}")

Standard Test Accuracy (Dropout OFF): 98.0600%


## Q3.2 Report the MC Dropout-enhanced accuracy (averaging 20 stochastic predictions).

In [10]:
def mc_dropout_predict(model, x, mc_runs=20):
    model.train()  # <— enable dropout at inference
    preds = []
    with torch.no_grad():
        for _ in range(mc_runs):
            logits = model(x)
            preds.append(torch.softmax(logits, dim=1))
    return torch.stack(preds).mean(0)  # average prediction


def evaluate_mc_dropout(model, mc_runs=20):
    correct = 0
    total = 0
    model.eval()  # we handle dropout manually inside function
    for x, y in test_loader:
        probs = mc_dropout_predict(model, x, mc_runs)
        pred = probs.argmax(1)
        correct += (pred == y).sum().item()
        total += y.size(0)
    return correct / total

mc_acc = evaluate_mc_dropout(model)
print(f"MC Dropout Test Accuracy (Dropout ON, 20 passes): {mc_acc:.4%}")

MC Dropout Test Accuracy (Dropout ON, 20 passes): 97.9800%


# Exercise-4: Transfer Learning with Pre-trained CNN
Use a pre-trained convolutional neural network (CNN) as a feature extractor and fine-
tune a classifier on a subset of the CIFAR-10 dataset. Set random seeds to 42. Follow
the configuration below:
* Load CIFAR-10 and normalize pixel values to [0,1]
* Use only the first 2000 training samples and first 500 test samples
* Load MobileNetV2 from tensorflow.keras.applications, with include top=False and weights=’imagenet’
* Freeze all layers of the pre-trained base
* Add a classifier on top:
    * GlobalAveragePooling2D
    * Dense layer with 128 neurons, ReLU activation
    * Dropout: 0.2
    * Output layer: 10 neurons with softmax
* Optimizer: Adam, learning rate = 0.001
* Loss: sparse categorical crossentropy
* epochs = 5, batch size = 32

In [15]:
# ---------------------------
# 1. CIFAR-10 dataset
# ---------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),    # MobileNet requires 224x224
    transforms.ToTensor(),            # Scales to [0,1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]), # ImageNet stats
])

train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_subset = Subset(train_dataset, range(2000))
test_subset = Subset(test_dataset, range(500))

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)

# ---------------------------
# 2. Load MobileNetV2, remove top, freeze base
# ---------------------------
mobilenet = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)

for p in mobilenet.features.parameters():  # freeze backbone only
    p.requires_grad = False

# Replace classifier to mimic Keras: GAP → Dense128 → Dropout → Dense10
# (BUT PyTorch already does GAP before classifier)
mobilenet.classifier = nn.Sequential(
    nn.Linear(1280, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 10)  # Softmax removed (CrossEntropyLoss handles it)
)

device = "cuda" if torch.cuda.is_available() else "cpu"
mobilenet.to(device)

# ---------------------------
# 3. Training setup
# ---------------------------
criterion = nn.CrossEntropyLoss()  # sparse categorical crossentropy
optimizer = optim.Adam(mobilenet.classifier.parameters(), lr=0.001)

# ---------------------------
# 4. Train
# ---------------------------
for epoch in range(5):
    mobilenet.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = mobilenet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/5, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1/5, Loss: 1.6089
Epoch 2/5, Loss: 1.0652
Epoch 3/5, Loss: 0.8992
Epoch 4/5, Loss: 0.7768
Epoch 5/5, Loss: 0.7176


## Q4.1 Report the test accuracy of the model

In [16]:
mobilenet.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = mobilenet(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Test Accuracy: {(correct/total)*100:.2f}%")

Test Accuracy: 69.40%


# Exercise-5: Deeper CNN Training on SVHN
Train a controlled deep convolutional neural network (CNN) on a subset of the SVHN
dataset. Set random seeds to 42. Load and preprocess SVHN. Build the network using
the following configuration:
* Load SVHN and normalize pixel values to [0,1]
* Use only the first 2000 training samples and first 500 test samples
* Input shape: 32 × 32 × 3
* CNN architecture:
  * Conv2D: 32 filters, 3×3 kernel, ReLU activation
  * Conv2D: 32 filters, 3×3 kernel, ReLU activation
  * MaxPooling2D: 2×2
  * Conv2D: 64 filters, 3×3 kernel, ReLU activation
  * Conv2D: 64 filters, 3×3 kernel, ReLU activation
  * MaxPooling2D: 2×2 Flatten
  * Dense: 256 neurons, ReLU activation
  * Dropout: 0.3
  * Output layer: 10 neurons with softmax
* Optimizer: Adam, learning rate = 0.001
* Loss: sparse categorical crossentropy
* epochs = 15, batch size = 32

In [17]:
# 1. Load SVHN (normalize to [0,1])

transform = transforms.ToTensor()  # already scales to [0,1]

train_data = datasets.SVHN(root="./data", split='train', download=True, transform=transform)
test_data  = datasets.SVHN(root="./data", split='test',  download=True, transform=transform)

# Use dataset subsets
train_subset = Subset(train_data, range(2000))
test_subset  = Subset(test_data, range(500))

batch_size = 32
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

# 2. Define CNN Model
class SVHNCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Linear(64*8*8, 256), nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 10)  # logits
        )

    def forward(self, x):
        return self.net(x)

model = SVHNCNN()

# 3. Optimizer & Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 4. Training loop
epochs = 15
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for x, y in train_loader:
        # SVHN labels are shape (N,1), squeeze needed
        y = y.long().squeeze()

        optimizer.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1:02d} | Loss: {total_loss/len(train_loader):.4f}")

print("Training complete.")

100%|██████████| 182M/182M [00:04<00:00, 41.8MB/s]
100%|██████████| 64.3M/64.3M [00:01<00:00, 36.4MB/s]


Epoch 01 | Loss: 2.2411
Epoch 02 | Loss: 2.2362
Epoch 03 | Loss: 2.2295
Epoch 04 | Loss: 2.2310
Epoch 05 | Loss: 2.2267
Epoch 06 | Loss: 2.2348
Epoch 07 | Loss: 2.2303
Epoch 08 | Loss: 2.2262
Epoch 09 | Loss: 2.2299
Epoch 10 | Loss: 2.2291
Epoch 11 | Loss: 2.2319
Epoch 12 | Loss: 2.2216
Epoch 13 | Loss: 2.2247
Epoch 14 | Loss: 2.2161
Epoch 15 | Loss: 2.1852
Training complete.


## Q5.1 Report the test accuracy of the deeper CNN model.

In [18]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for x, y in test_loader:
        y = y.long().squeeze()
        preds = model(x).argmax(1)
        correct += (preds == y).sum().item()
        total += y.size(0)

print(f"Test Accuracy: {correct/total:.4%}")

Test Accuracy: 19.2000%


# Exercise-6: CNN with SGD, MC Dropout, and Epistemic Uncertainty
Train a controlled convolutional neural network (CNN) on a subset of the SVHN dataset
using SGD optimizer. Then, apply Monte Carlo (MC) Dropout at inference to estimate
both test accuracy and epistemic uncertainty. Set random seeds to 42. Use the following
configuration:
* Load SVHN and normalize pixel values to [0,1]
* Use only the first 2000 training samples and first 500 test samples
* Input shape: 32 × 32 × 3
* CNN architecture:
  * Conv2D: 32 filters, 3×3 kernel, ReLU activation
  * MaxPooling2D: 2×2
  * Conv2D: 64 filters, 3×3 kernel, ReLU activation
  * MaxPooling2D: 2×2
  * Flatten
  * Dense: 128 neurons, ReLU activation
  * Dropout: 0.25 (keep during inference for MC Dropout)
  * Output layer: 10 neurons with softmax
* Optimizer: SGD with momentum = 0.9, learning rate = 0.01
* Loss: sparse categorical crossentropy
* epochs = 15, batch size = 32
* For MC Dropout:
  * Enable dropout during inference
  * Average predictions over 20 stochastic forward passes
  * Compute the epistemic uncertainty as the predictive variance across passes

In [19]:
# 1. Load SVHN (normalize to [0,1])
transform = transforms.ToTensor()

train_data = datasets.SVHN(root="./data", split='train', download=True, transform=transform)
test_data  = datasets.SVHN(root="./data", split='test',  download=True, transform=transform)

train_subset = Subset(train_data, range(2000))
test_subset  = Subset(test_data, range(500))

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_subset, batch_size=32, shuffle=False)

# 2. CNN Model
class MC_Dropout_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Linear(64*8*8, 128), nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(128, 10)  # logits
        )

    def forward(self, x):
        return self.net(x)

model = MC_Dropout_CNN()

# 3. Optimizer & Loss
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()

# 4. Training
epochs = 15
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for x, y in train_loader:
        y = y.long().squeeze()
        optimizer.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1:02d} | Loss: {total_loss/len(train_loader):.4f}")

print("Training complete.\n")

Epoch 01 | Loss: 2.2465
Epoch 02 | Loss: 2.2277
Epoch 03 | Loss: 2.2278
Epoch 04 | Loss: 2.2226
Epoch 05 | Loss: 2.2207
Epoch 06 | Loss: 2.2113
Epoch 07 | Loss: 2.2043
Epoch 08 | Loss: 2.1907
Epoch 09 | Loss: 2.1619
Epoch 10 | Loss: 2.1213
Epoch 11 | Loss: 2.0294
Epoch 12 | Loss: 1.8501
Epoch 13 | Loss: 1.6025
Epoch 14 | Loss: 1.3268
Epoch 15 | Loss: 1.1186
Training complete.



## Q6.1 Report the plain test accuracy of the CNN trained with SGD (no MC Dropout).

In [20]:
def eval_standard(model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for x, y in test_loader:
            y = y.long().squeeze()
            preds = model(x).argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return correct / total

base_acc = eval_standard(model)
print(f"Standard Test Accuracy: {base_acc:.4%}")

Standard Test Accuracy: 58.8000%


## Q6.2 Report the MC Dropout-enhanced accuracy (averaging 20 stochastic predictions).

In [21]:
def mc_dropout_predict(model, x, mc_runs=20):
    model.train()  # enable dropout for MC inference
    preds = []
    with torch.no_grad():
        for _ in range(mc_runs):
            logits = model(x)
            preds.append(torch.softmax(logits, dim=1))
    return torch.stack(preds)  # shape: [mc_runs, batch, 10]

def eval_mc_dropout(model, mc_runs=20):
    correct, total = 0, 0
    uncertainties = []

    for x, y in test_loader:
        y = y.long().squeeze()

        mc_preds = mc_dropout_predict(model, x, mc_runs)
        mean_probs = mc_preds.mean(dim=0)  # mean prediction
        var_probs = mc_preds.var(dim=0).mean(dim=1)  # avg var per sample (epistemic)

        preds = mean_probs.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

        uncertainties.extend(var_probs.cpu().numpy())

    return (correct / total), np.array(uncertainties)

mc_acc, epistemic_unc = eval_mc_dropout(model, mc_runs=20)

print(f"MC Dropout Test Accuracy: {mc_acc:.4%}")

MC Dropout Test Accuracy: 59.0000%


## Q6.3 Compute the average epistemic uncertainty (mean predictive variance) across all test samples. Report it as a deterministic number rounded to 3 decimal places

In [23]:
print(f"Mean Epistemic Uncertainty: {epistemic_unc.mean():.6f}")
print(f"Uncertainty Std: {epistemic_unc.std():.6f}")

Mean Epistemic Uncertainty: 0.003791
Uncertainty Std: 0.003380
